In [1]:
# Import Dependencies
import pandas as pd
import re
import numpy as np

In [2]:
# Load the dataset
file_path = "./winemag-data-130k-v2.csv"
wine_df = pd.read_csv(file_path)

In [3]:
# Initial exploration
print(wine_df.info())
print(wine_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129971 entries, 0 to 129970
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   wine_no         129971 non-null  int64  
 1   country         129908 non-null  object 
 2   description     129971 non-null  object 
 3   designation     92506 non-null   object 
 4   points          129971 non-null  int64  
 5   price           120975 non-null  float64
 6   province        129908 non-null  object 
 7   region_1        108724 non-null  object 
 8   region_2        50511 non-null   object 
 9   taster_name     103727 non-null  object 
 10  taster_twitter  98758 non-null   object 
 11  title           129971 non-null  object 
 12  variety         129970 non-null  object 
 13  winery          129971 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 13.9+ MB
None
   wine_no   country                                        description  \
0        

In [4]:
# Extract the year from the title
def extract_year(title):
    match = re.search(r"\b(19\d{2}|20\d{2})\b", str(title))
    return match.group(0) if match else np.nan

wine_df['vintage_year'] = wine_df['title'].apply(extract_year)
print(wine_df[['title', 'vintage_year']].head())

                                               title vintage_year
0                  Nicosia 2013 Vulkà Bianco  (Etna)         2013
1      Quinta dos Avidagos 2011 Avidagos Red (Douro)         2011
2      Rainstorm 2013 Pinot Gris (Willamette Valley)         2013
3  St. Julian 2013 Reserve Late Harvest Riesling ...         2013
4  Sweet Cheeks 2012 Vintner's Reserve Wild Child...         2012


In [5]:
# Categorize points into quality levels
def categorize_points(score):
    if pd.isnull(score):
        return 'unknown'
    score = int(score)
    if score >= 95:
        return 'excellent'
    elif score >= 90:
        return 'very good'
    elif score >= 85:
        return 'good'
    elif score >= 80:
        return 'average'
    else:
        return 'below average'

wine_df['rating_category'] = wine_df['points'].apply(categorize_points)
print(wine_df[['points', 'rating_category']].head())

   points rating_category
0      87            good
1      87            good
2      87            good
3      87            good
4      87            good


In [6]:
# Simplify wine type detection based on variety
def identify_wine_style(variety):
    variety = str(variety).lower()
    if any(x in variety for x in ['red', 'cabernet', 'merlot', 'pinot noir', 'zinfandel']):
        return 'red'
    elif any(x in variety for x in ['white', 'chardonnay', 'sauvignon', 'riesling']):
        return 'white'
    elif 'rosé' in variety or 'rose' in variety:
        return 'rosé'
    else:
        return 'unknown'

wine_df['style'] = wine_df['variety'].apply(identify_wine_style)
print(wine_df[['variety', 'style']].head())

          variety    style
0     White Blend    white
1  Portuguese Red      red
2      Pinot Gris  unknown
3        Riesling    white
4      Pinot Noir      red


In [7]:
# Replace null values in region_1 with province
wine_df['region_1'].fillna(wine_df['province'], inplace=True)
print(wine_df[['province', 'region_1']].head())

            province             region_1
0  Sicily & Sardinia                 Etna
1              Douro                Douro
2             Oregon    Willamette Valley
3           Michigan  Lake Michigan Shore
4             Oregon    Willamette Valley


C:\Users\Alyssandra_Calhoun\AppData\Local\Temp\ipykernel_48092\2332221452.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  wine_df['region_1'].fillna(wine_df['province'], inplace=True)


In [8]:
# Handle missing taster names
wine_df['taster_name'].fillna('unknown', inplace=True)
print(wine_df[['taster_name']].head())

          taster_name
0       Kerin O’Keefe
1          Roger Voss
2        Paul Gregutt
3  Alexander Peartree
4        Paul Gregutt


C:\Users\Alyssandra_Calhoun\AppData\Local\Temp\ipykernel_48092\3598122313.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  wine_df['taster_name'].fillna('unknown', inplace=True)


In [9]:
# Select and reorder columns for the clean dataset
refined_wine_df = wine_df[['country', 'points', 'price', 'province', 'region_1', 'taster_name',
                           'title', 'variety', 'winery', 'vintage_year', 'style', 'rating_category']]

# Check for missing values
def check_missing_values(df):
    missing_data = df.isnull().sum()
    missing_percent = (missing_data / len(df)) * 100
    missing_summary = pd.DataFrame({
        'Missing Values': missing_data,
        '% of Total': missing_percent.round(1)
    })
    missing_summary = missing_summary[missing_summary['Missing Values'] > 0]
    print(missing_summary)

check_missing_values(refined_wine_df)

              Missing Values  % of Total
country                   63         0.0
price                   8996         6.9
province                  63         0.0
region_1                  63         0.0
variety                    1         0.0
vintage_year            4626         3.6


In [10]:
# Drop rows with any remaining null values
refined_wine_df.dropna(inplace=True)
refined_wine_df.reset_index(drop=True, inplace=True)
print(refined_wine_df.head())

    country  points  price        province             region_1  \
0  Portugal      87   15.0           Douro                Douro   
1        US      87   14.0          Oregon    Willamette Valley   
2        US      87   13.0        Michigan  Lake Michigan Shore   
3        US      87   65.0          Oregon    Willamette Valley   
4     Spain      87   15.0  Northern Spain              Navarra   

          taster_name                                              title  \
0          Roger Voss      Quinta dos Avidagos 2011 Avidagos Red (Douro)   
1        Paul Gregutt      Rainstorm 2013 Pinot Gris (Willamette Valley)   
2  Alexander Peartree  St. Julian 2013 Reserve Late Harvest Riesling ...   
3        Paul Gregutt  Sweet Cheeks 2012 Vintner's Reserve Wild Child...   
4   Michael Schachner  Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...   

              variety               winery vintage_year    style  \
0      Portuguese Red  Quinta dos Avidagos         2011      red   
1   

C:\Users\Alyssandra_Calhoun\AppData\Local\Temp\ipykernel_48092\2481559817.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  refined_wine_df.dropna(inplace=True)


In [11]:
# Save the finalized clean dataset
final_clean_file_path = "./winemag-data-clean.csv"
refined_wine_df.to_csv(final_clean_file_path, index=False)

print(f"Final cleaned data saved to {final_clean_file_path}")

Final cleaned data saved to ./winemag-data-clean.csv


In [12]:
# Final summary
print(refined_wine_df.info())
print(refined_wine_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116765 entries, 0 to 116764
Data columns (total 12 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   country          116765 non-null  object 
 1   points           116765 non-null  int64  
 2   price            116765 non-null  float64
 3   province         116765 non-null  object 
 4   region_1         116765 non-null  object 
 5   taster_name      116765 non-null  object 
 6   title            116765 non-null  object 
 7   variety          116765 non-null  object 
 8   winery           116765 non-null  object 
 9   vintage_year     116765 non-null  object 
 10  style            116765 non-null  object 
 11  rating_category  116765 non-null  object 
dtypes: float64(1), int64(1), object(10)
memory usage: 10.7+ MB
None
    country  points  price        province             region_1  \
0  Portugal      87   15.0           Douro                Douro   
1        US      87   14.0  